<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Jogadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title

import numpy as np
import pandas as pd
import requests
import asyncio
from datetime import date
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns
from matplotlib import rcParams
from google.colab import data_table

In [2]:
#@title

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 100)
rcParams['figure.figsize'] = 20,9.27
rcParams['axes.titlesize'] = 22
rcParams['axes.labelsize'] = 16
data_table.enable_dataframe_formatter()
np.random.seed(777)

#1 - Parametrização



In [3]:
#@title

leagues = requests.get('https://api.sofascore.com/api/v1/config/unique-tournaments/BR/football').json()['uniqueTournaments']
seasons = []

In [8]:
#@title

dp_leagues_data = []
dp_seasons_data = []
for l in leagues:  
  dp_leagues_data.append((l['name'], l['id']))
  s = requests.get(f'https://api.sofascore.com/api/v1/unique-tournament/{l["id"]}/seasons').json()['seasons']  
  for si in s:
    si['league'] = l['id']
    dp_seasons_data.append((si['year'], si['id'], si['league']))      

No menu abaixo selecione as ligas e temporadas que deseja obter os dados.

In [9]:
#@title

#UI
add_button = widgets.Button(description="Adicionar")
clear_button = widgets.Button(description="Limpar Seleções")
dp_leagues = widgets.Dropdown(options=dp_leagues_data, value=None)
dp_seasons = widgets.Dropdown(value=None)
UI_INPUTS = widgets.HBox([dp_leagues, dp_seasons])
UI_BUTTONS = widgets.HBox([add_button, clear_button])
output = widgets.Output()
selected_leagues = []
selected_seasons = []

def on_change(event):
  dp_seasons_league = []
  if event['name'] == 'value':
    for ds in dp_seasons_data:
      #Busco apenas pelas seasons da liga selecionada pela posição na tupla
      if ds[2] == event['new']: 
        dp_seasons_league.append(ds[0:2])
    dp_seasons.options = dp_seasons_league

def add_league(b):
  if dp_leagues.value == None or dp_seasons.value == None:
    output.clear_output()
    print('Informações não preenchidas!')
  elif dp_leagues.value in selected_leagues and dp_seasons.value in selected_seasons:
    with output:      
      output.clear_output()
      print(f"Erro: Liga e temporada já adicionadas")       
  else:
    selected_leagues.append(dp_leagues.value)
    selected_seasons.append(dp_seasons.value) 
    output.clear_output()   
    with output:      
      print('Liga adicionada com sucesso! :D')

def clear_selection(b):
  selected_leagues.clear()
  selected_seasons.clear()
  with output:      
      print('Parâmetros reiniciados.')
      
dp_leagues.observe(on_change)
add_button.on_click(add_league)
clear_button.on_click(clear_selection)
display(UI_INPUTS, UI_BUTTONS, output)

Output()

Agora selecione as posições dos jogadores que terão seus dados extraídos.

*Para selecionar mais de uma posição use as teclas CRTL ou SHIFT*

In [10]:
#@title

dp_positions = widgets.SelectMultiple(
    options=[('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), ('Atacantes','F')],    
    value=['G', 'D', 'M', 'F'],
    #rows=10,
    description='Posições: ',
    disabled=False
)
display(dp_positions)

SelectMultiple(description='Posições: ', index=(0, 1, 2, 3), options=(('Goleiros', 'G'), ('Defensores', 'D'), …

#2 - Obtenção das estatisticas
Nesta etapa iremos utilizar os parâmetros definidos acima para capturar os dados no SofaScore

In [11]:
#@title
def make_url_stats(competition, season, position, offset):
  return f"https://api.sofascore.com/api/v1/unique-tournament/{competition}/season/{season}/statistics?offset={offset}&order=-rating&accumulation=total&filters=position.in.{position}&fields=%2CbigChancesMissed%2CsuccessfulDribbles%2CsuccessfulDribblesPercentage%2CtotalShots%2CshotsOnTarget%2CshotsOffTarget%2CblockedShots%2CgoalConversionPercentage%2CpenaltiesTaken%2CpenaltyGoals%2CpenaltyWon%2CshotFromSetPiece%2CfreeKickGoal%2CgoalsFromInsideTheBox%2CgoalsFromOutsideTheBox%2CheadedGoals%2CleftFootGoals%2CrightFootGoal%2CshitWoodwork%2Coffsides%2CpenaltyConversion%2CsetPieceConversion%2Crating%2Cinterceptions%2CpenaltyConceded%2Cclearances%2CerrorLeadToGoal%2CerrorLeadToShot%2CownGoals%2CdribbledPast%2CcleanSheet%2C rating %2CbigChancesCreated%2Cassists %2CaccuratePasses%2CinaccuratePasses%2CtotalPasses%2CaccuratePassesPercentage %2CaccurateOwnHalfPasses%2CaccurateOppositionHalfPasses%2CaccurateFinalThirdPasses%2CkeyPasses%2CaccurateCrosses%2CaccurateCrossesPercentage%2CaccurateLongBalls%2CaccurateLongBallsPercentage%2CpassToAssist%2Crating %2CcleanSheet%2CpenaltyFaced%2CpenaltySave%2CsavedShotsFromInsideTheBox%2CsavedShotsFromOutsideTheBox%2CgoalsConcededInsideTheBox%2CgoalsConcededOutsideTheBox%2Cpunches%2CsuccessfulRunsOut%2ChighClaims%2CcrossesNotClaimed%2Crating %2CyellowCards%2CredCards%2CgroundDuelsWon%2CgroundDuelsWonPercentage%2CaerialDuelsWon%2CaerialDuelsWonPercentage%2CtotalDuelsWon%2CtotalDuelsWonPercentage%2CminutesPlayed%2CwasFouled%2Cfouls%2Cdispossessed%2CpossessionLost%2Cappearances%2CmatchesStarted%2Crating &limit=20"

In [12]:
#@title
def get_stats(leagues, seasons, positions):
  stats = pd.DataFrame()  
  for idx, l in enumerate(leagues):    
    for p in positions:
     iw = 0
     while True:  
       response = requests.get(make_url_stats(l, seasons[idx], p, iw))
       #Já percoreu todos os players então paro
       if response.json()['results'] == []:        
         break;

       response = response.json()['results']
       response = pd.json_normalize(response, max_level=6)
       response['league'] = l      
       stats = pd.concat([stats, response], ignore_index=True)    
       iw = iw + 20 
  return stats

In [13]:
#@title
df_players = get_stats(selected_leagues, selected_seasons, dp_positions.value)

#3 - Enriquecimento dos dados
Na etapa anterior obtivemos todas as estatisticas disponíveis para os jogadores, agora vamos fazer o enriquecimento desses dados com outras informações.

Buscando por posições alternativas para cada player, bem como algumas caracteristicas, como altura, valor de mercado e etc...

In [14]:
async def parallel_requests(coroutines):
  await asyncio.gather(*coroutines)

In [15]:
players_atributtes = []
players_alternative_positions = []

async def get_player_attributes(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}')
  response = response_data.json()['player']
  response['player.id'] = player_id #Para fazer o merge depois
  players_atributtes.append(response)

async def get_alternative_positions(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}/characteristics')
  response = response_data.json()
  if len(response['positions']) >=1:
    response = {key: response[key] for key in response.keys() if key == 'positions'}
    response['player.id'] = player_id #Para fazer o merge depois
    players_alternative_positions.append(response)

In [16]:
req_attributes = [get_player_attributes(p) for p in df_players['player.id']]
req_alternative_positions = [get_alternative_positions(p) for p in df_players['player.id']]

In [17]:
await parallel_requests(req_alternative_positions)

In [18]:
await parallel_requests(req_attributes)

In [19]:
df_players_attributes = pd.json_normalize(players_atributtes, max_level=2)
df_players_alternative_positions = pd.json_normalize(players_alternative_positions, max_level=2)

In [20]:
merged_data = df_players_alternative_positions.merge(df_players_attributes, on='player.id')

In [21]:
#Como vieram muitas informações redundantes iremos dropá-las
merged_data = merged_data.drop(columns=['name', 'firstName', 'lastName', 'slug', 'shortName','proposedMarketValue', 'jerseyNumber','team.slug', 'team.shortName', 'team.gender', 'team.sport.name',
       'team.sport.slug', 'team.sport.id', 'team.tournament.name',
       'team.tournament.slug', 'team.tournament.category',
       'team.tournament.uniqueTournament', 'team.tournament.priority',
       'team.tournament.id', 'team.primaryUniqueTournament.name',
       'team.primaryUniqueTournament.slug',
       'team.primaryUniqueTournament.category',
       'team.primaryUniqueTournament.userCount',
       'team.primaryUniqueTournament.id',
       'team.primaryUniqueTournament.displayInverseHomeAwayTeams',
       'team.userCount', 'team.nameCode', 'team.disabled', 'team.national',
       'team.type', 'team.id', 'team.country.alpha2', 'team.country.name',
       'team.teamColors.primary', 'team.teamColors.secondary',
       'team.teamColors.text', 'userCount', 'id', 'team.name'])

In [22]:
merged_data.rename(columns={'country.alpha2':'countryCode', 'country.name':'countryName', 'proposedMarketValueRaw.value':'proposedMarketValue', 'proposedMarketValueRaw.currency': 'proposedMarketCurrency'}, inplace=True)

In [23]:
df_players = merged_data.merge(df_players, on='player.id')

## 3.2 - Engenharia reversa em features percentuais

Para alguns fundamentos é exibida a quantidade e o percentual de acertos do jogador, mas não há a infomação de quantos quandos lances ele errou. 

Para o nosso modelo é mais interessante termos os dados dos acertos e dos erros do que os percentuais, por isso usei a regra de 3 para criar novas features demonstrando os erros cometidos em cada fundamento

In [24]:
def reverse_feature(frame, number, percentage, method = np.ceil):
 return ((frame[number] / (frame[percentage]/100)) - frame[number]).fillna(0).apply(lambda x: int(method(x)))

In [25]:
df_players['aerialDuelsLost'] = reverse_feature(df_players, 'aerialDuelsWon', 
                                                'aerialDuelsWonPercentage')

In [26]:
df_players['groundDuelsLost'] = reverse_feature(df_players, 'groundDuelsWon', 
                                              'groundDuelsWonPercentage')

In [27]:
df_players['unsuccessfulDribbles'] = reverse_feature(df_players, 'successfulDribbles', 
                                                     'successfulDribblesPercentage', np.ceil)

In [28]:
df_players['unnacurateLongBalls'] = reverse_feature(df_players, 'accurateLongBalls',
                                                    'accurateLongBallsPercentage')

In [29]:
df_players['unaccurateCrosses'] = reverse_feature(df_players, 'accurateCrosses', 
                                                            'accurateCrossesPercentage')

In [30]:
df_players['successfulShotFromSetPiece'] = (df_players['shotFromSetPiece'] * (df_players['setPieceConversion']/100)).apply(lambda x: int(np.ceil(x)))

In [31]:
df_players['unsuccessfulShotFromSetPiece'] = reverse_feature(df_players, 'successfulShotFromSetPiece',
                                                             'setPieceConversion', np.floor)

#4 - Tratamento dos dados

##4.1 - Tratando valores nulos

In [32]:
df_players.columns[df_players.isna().any()].tolist()

['retired',
 'shirtNumber',
 'contractUntilTimestamp',
 'proposedMarketValue',
 'proposedMarketCurrency']

Para o prazo de término dos contratos dos jogadores iremos assumir que quem não possuir esta informação preenchida irá ter o seu contrato encerrado no fim da próxima temporada.

In [33]:
from datetime import date
import time
next_year = date.today().year + 1
next_season_end = date(next_year, 12,31)
next_season_end = time.mktime(next_season_end.timetuple())

In [34]:
df_players.fillna({'height': np.ceil(df_players['height'].mean()),
                   'preferredFoot': df_players['preferredFoot'].mode().values[0], 
                   'shirtNumber': 0,
                   'retired': False,
                   'proposedMarketCurrency': 'EUR',
                   'contractUntilTimestamp': next_season_end
                   }, inplace=True)

In [35]:
df_players['proposedMarketValue'].dropna(inplace=True)

In [36]:
df_players = df_players[df_players.columns.sort_values(ascending=True)]

In [37]:
#Só quero players que estiveram em alguma partida na temporada
df_players = df_players[df_players['proposedMarketValue']>0]

##4.2 - Tratamento de transferências
Como é possível capturar dados de ligas diferentes, pode ser que o jogador que estava em uma Liga A tenha se transferido para uma Liga B, com isso o seu registro é exibido duas vezes o que irá causar um problema na execução do algoritmo KNN já que em breve iremos indexar o dataframe pelo ID do Jogador. Portanto, optei por eliminar os jogadores duplicados já que não faz tanto sentido comparar ele com ele mesmo.

In [38]:
df_players = df_players.drop_duplicates(subset=['player.id'], keep='last', ignore_index=True)

## 4.3 - Tratamento de Outliers [*TODO*]

#5 - Análise exploratória  [*TODO*]



#6 - Geração do Modelo

##6.1 - Transformando as posições alternativas em colunas

In [39]:
X = df_players.copy()

In [40]:
X[['AM', 'DC', 'DL', 'DM', 'DR', 'LW', 'MC', 'ML', 'MR', 'RW', 'ST']] = 0

In [41]:
for i, p in enumerate(X['positions'].tolist()):
    X.loc[i, p] = 1

In [42]:
#Alterando o indice para o player.id
X.set_index('player.id', inplace = True)

##6.2 - Enconding das features categoricas

In [43]:
from sklearn.preprocessing import OneHotEncoder
f_encoder = OneHotEncoder()
c_encoder = OneHotEncoder()
p_encoder = OneHotEncoder()
feet = f_encoder.fit_transform(X[['preferredFoot']])
country = c_encoder.fit_transform(X[['countryName']])
position = p_encoder.fit_transform(X[['position']])

In [44]:
X[f_encoder.categories_[0]] = feet.toarray()
X[c_encoder.categories_[0]] = country.toarray()
X[p_encoder.categories_[0]] = position.toarray()

In [45]:
X.drop([
        'player.name', 'player.slug',
       'player.userCount', 'team.name', 'team.slug', 'team.shortName',
       'team.sport.name', 'team.sport.slug', 'team.sport.id', 'team.userCount',
       'team.type', 'team.id', 'team.teamColors.primary',
       'team.teamColors.secondary', 'team.teamColors.text', 'shirtNumber', 'league',
       'positions','position', 'preferredFoot', 'shirtNumber',
        'countryCode', 'countryName',
        'retired','proposedMarketCurrency',
       #Percentuais já foram convertidos em novas features anteriormente
       'accurateLongBallsPercentage','aerialDuelsWonPercentage','goalConversionPercentage','groundDuelsWonPercentage',
       'successfulDribblesPercentage','totalDuelsWonPercentage', 'setPieceConversion', 'accurateCrossesPercentage',
       #Redundantes em relação aos erros e acertos
       'totalDuelsWon', 'totalShots', 'totalPasses', 'shotFromSetPiece'
], axis='columns', inplace=True)

In [46]:
from scipy.sparse import csr_matrix
stats_sparse = csr_matrix(X)

In [47]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(stats_sparse)

NearestNeighbors(algorithm='brute')

# 7 - Apresentação dos Resultados

In [48]:
#@title
from ipywidgets import interact

In [49]:
#@title
dp_players = df_players[['player.name','player.id']].sort_values(by=['player.name']).values.tolist()

In [50]:
#@title
def get_similars(Player):
  distance, similar_players_index = model.kneighbors(X.loc[Player].values.reshape(1,-1))
  similar_players = pd.DataFrame()
  similar_players = df_players.loc[df_players.index[similar_players_index[0]]]
  similar_players['distances'] = distance[0]
  return similar_players[['player.name', 'team.name','rating']]

In [51]:
#@title
interact(get_similars, Player=dp_players);

interactive(children=(Dropdown(description='Player', options=(['Ademir', 922546], ['Adil Aouchiche', 962877], …